### In Case of Using CoLab : 구글드라이브 연계

In [17]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


★ setting point: Set the folder path where your data is stored.

example: "/content/gdrive/MyDrive/Colab Notebooks/llama/01. data preparation/dataset"


In [18]:
dataPath = "/content/gdrive/MyDrive/LLMClass/datasets/"
#dataPath = "./dataset/"
!ls /content/gdrive/MyDrive/LLMStudy/datasets/

ls: cannot access '/content/gdrive/MyDrive/LLMStudy/datasets/': No such file or directory


### Install Library

In [21]:
# datasets: loads and manages data from hugging-face's community.
# jsonlines: handles json format data
!pip install datasets==2.16.1 jsonlines==4.0.0

### Login to access huggign face

In [ ]:
# How to check HF access token: hugging face -> select account at top right -> settings -> access token
import huggingface_hub
huggingface_hub.login()

### Colab 에서 huggingface hub 로그인이 잘 안되면 아래와 같이 cli login 으로 진행

In [20]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: write

In [ ]:
# Define installed libraries

In [23]:
import pandas as pd
import json
import jsonlines
from datasets import Dataset
import os

# ★ Hugging face 없이 csv 바로 튜닝데이터로 변환 ★ HF 호환

In [24]:
!ls /content/gdrive/MyDrive/LLMClass/datasets/btsdata_resp.csv

/content/gdrive/MyDrive/LLMClass/datasets/btsdata_resp.csv


In [25]:
csvfilepath = dataPath+"btsdata_resp.csv"
jsonfilepath = dataPath+"btsdata_resp.json"


In [26]:
import pandas as pd

In [27]:
#csvdata = pd.read_csv(csvfilepath, encoding="ms949")
try:
    csvdata = pd.read_csv(csvfilepath, encoding="ms949")
except Exception as e:
    csvdata = pd.read_csv(csvfilepath, encoding="utf-8")

data_json = csvdata.to_json(jsonfilepath)

In [28]:
df = pd.read_json(jsonfilepath)

In [29]:
df.head(5)

,instruction,response
0,BTS는 누구인가요?,2013년 6월 13일에 데뷔한 빅히트 뮤직 소속의 7인조 대한민국 국적 보이그룹입니다.
1,BTS의 특징은 무엇입니까,"그들의 특징은 팝, 알앤비, 힙합을 아우르는 음악 스타일과 정교하고 절도 있는 안무..."
2,BTS의 메시지는 무엇인가요?,"세상의 불평등과 폭력을 용인하지 말고, 자신을 사랑하고, 더 나은 세상을 위해 함께..."
3,BTS의 응원도구는 무엇인가요?,아미밤 입니다.
4,방탄소년단의 멤버 수는 모두 몇 명인가?,대한민국 출신의 7명의 멤버로 구성된 방탄소년단은 공식적으로 2013년 세계 무대에...


In [30]:
from datasets import Dataset

def format_example(row):
    return {
        'text': f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>

        You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>

        {row['instruction']}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>

        {row['response']}<|eot_id|>"""
    }

# 판다스 데이터프레임을 데이터셋으로 변환하고, 포맷팅 함수 적용
dataset = Dataset.from_pandas(df)
dataset = dataset.map(format_example)

Map:   0%|          | 0/146 [00:00<?, ? examples/s]

In [31]:
dataset[2]

{'instruction': 'BTS의 메시지는 무엇인가요?',
 'response': '세상의 불평등과 폭력을 용인하지 말고, 자신을 사랑하고, 더 나은 세상을 위해 함께하자는 것이 BTS의 메시지입니다.',
 '__index_level_0__': 2,
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n        You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\n        BTS의 메시지는 무엇인가요?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\n        세상의 불평등과 폭력을 용인하지 말고, 자신을 사랑하고, 더 나은 세상을 위해 함께하자는 것이 BTS의 메시지입니다.<|eot_id|>'}

In [32]:
# 필요시 허깅페이스에 업로드
dataset.push_to_hub("rssaem/btsdata_resp")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/432 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rssaem/btsdata_resp/commit/d0debb4b090399f869f4ae65311608fcb877f380', commit_message='Upload dataset', commit_description='', oid='d0debb4b090399f869f4ae65311608fcb877f380', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/rssaem/btsdata_resp', endpoint='https://huggingface.co', repo_type='dataset', repo_id='rssaem/btsdata_resp'), pr_revision=None, pr_num=None)